In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [2]:
import pydot

In [3]:
with open('E://cnews/cnews.vocab.txt', encoding='utf8') as file:
    vocabulary_list = [k.strip() for k in file.readlines()]
#读取词表
with open('E://cnews/cnews.train.txt', encoding='utf8') as file:
    line_list = [k.strip() for k in file.readlines()]
    #读取每行
    train_label_list = [k.split()[0] for k in line_list]
    #将标签依次取出
    train_content_list = [k.split(maxsplit=1)[1] for k in line_list]
    #将内容依次取出,此处注意split()选择最大分割次数为1,否则句子被打断.
#同理读取test数据
with open('E://cnews/cnews.test.txt', encoding='utf8') as file:
    line_list = [k.strip() for k in file.readlines()]
    test_label_list = [k.split()[0] for k in line_list]
    test_content_list = [k.split(maxsplit=1)[1] for k in line_list]

In [4]:
word2id_dict = dict(((b, a) for a, b in enumerate(vocabulary_list)))
def content2vector(content_list):
    content_vector_list = []
    for content in content_list:
        content_vector = []
        for word in content:
            if word in word2id_dict:
                content_vector.append(word2id_dict[word])
            else:
                content_vector.append(word2id_dict['<PAD>'])
        content_vector_list.append(content_vector)
    return content_vector_list
train_vector_list = content2vector(train_content_list)
test_vector_list = content2vector(test_content_list)

In [5]:
import keras as kr
num_classes=10
maxlen=600

Using TensorFlow backend.


In [6]:
label = LabelEncoder()
train_Y = kr.utils.to_categorical(label.fit_transform(train_label_list),num_classes=num_classes)
test_Y = kr.utils.to_categorical(label.fit_transform(test_label_list),num_classes=num_classes)

In [7]:
from keras.preprocessing.sequence import pad_sequences

In [8]:
train_X = pad_sequences(train_vector_list,maxlen=600,truncating='post',padding='post') # 和原作者的不同
test_X = pad_sequences(test_vector_list,maxlen=600,truncating='post',padding='post') # 和原作者不同 

In [9]:
from keras.models import Sequential 
from keras import layers

In [10]:
model = Sequential()
Embdding=model.add(layers.Embedding(5000,128,input_length=600))
con=model.add(layers.Conv1D(filters=256,kernel_size=8))
maxpool=model.add(layers.MaxPooling1D(593))
flatten=model.add(layers.Flatten())
dense1=model.add(layers.Dense(128, input_dim=100))  # 64是输出层的维度
dropout=model.add(layers.Dropout(0.2))  # 控制需要断开的神经元比例，此处应该为0.2
active1=model.add(layers.core.Activation('relu'))
dense2=model.add(layers.Dense(10, input_shape=(600,128)))
predict_y=model.add(layers.core.Activation('softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 600, 128)          640000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 593, 256)          262400    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 1, 256)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
activation_1 (Activation)    (None, 128)               0         
__________

In [11]:
from keras import optimizers
optimizer = optimizers.Adam(lr=0.001)

In [12]:
model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['categorical_accuracy'] )

In [13]:
len(train_Y)

50000

In [14]:
import random
for i in range(1000):
    train_index = random.sample(list(range(len(train_Y))),k=64)
    X = train_X[train_index]
    Y = train_Y[train_index]
    model.fit(X,Y,verbose=0)
    step = i + 1 
    if step % 100 == 0:
        test_index = random.sample(list(range(len(test_Y))), k=200)
        x = test_X[test_index]
        y = test_Y[test_index]
        loss_value, accuracy_value = model.evaluate(x,y,verbose=0)
        print('step:%d loss:%.4f accuracy:%.4f' %(step, loss_value, accuracy_value))

step:100 loss:0.5100 accuracy:0.8550
step:200 loss:0.3632 accuracy:0.8850
step:300 loss:0.1908 accuracy:0.9300
step:400 loss:0.1505 accuracy:0.9650
step:500 loss:0.1947 accuracy:0.9450
step:600 loss:0.1246 accuracy:0.9700
step:700 loss:0.1799 accuracy:0.9550
step:800 loss:0.1554 accuracy:0.9600
step:900 loss:0.1190 accuracy:0.9500
step:1000 loss:0.1465 accuracy:0.9650


In [16]:
from keras.utils import plot_model
plot_model(model,to_file='model.png')